## 1. Functional Similarity (GO SLIM)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import re
from typing import Dict, Tuple, Optional
import logging

# Configuration des chemins
BASE_DIR = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data")
RAW_DATA_DIR = BASE_DIR / "raw data" / "autres"
CLEAN_DATA_DIR = BASE_DIR / "clean data"
INTERACTIONS_DIR = CLEAN_DATA_DIR / "interactions"
OUTPUT_DIR = CLEAN_DATA_DIR / "autres"
GO_SLIM_YEAST = RAW_DATA_DIR / "go_slim_mapping.tab"
GO_SLIM_HUMAN = RAW_DATA_DIR / "uniprotkb_Homo_sapiens_Human_AND_model_2025_04_03.tsv"
MAPPING_FILE = RAW_DATA_DIR / "YEAST_559292_idmapping.dat"

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def load_mapping(mapping_file: Path) -> Tuple[Dict[str, str], Dict[str, str]]:
    """Charge le mapping UniProt vers SGD et inversement"""
    try:
        dtype = {'UniProt': 'string', 'DB': 'string', 'ID': 'string'}
        df = pd.read_csv(mapping_file, sep='\t', header=None,
                        names=['UniProt', 'DB', 'ID'], dtype=dtype)
        
        sgd_mapping = df[df['DB'] == 'SGD']
        uniprot_to_sgd = dict(zip(sgd_mapping['UniProt'], sgd_mapping['ID']))
        sgd_to_uniprot = dict(zip(sgd_mapping['ID'], sgd_mapping['UniProt']))
        
        logger.info(f"Loaded {len(uniprot_to_sgd)} UniProt to SGD mappings")
        return uniprot_to_sgd, sgd_to_uniprot
    except Exception as e:
        logger.error(f"Erreur lors du chargement du mapping: {e}")
        return {}, {}

def load_go_slim(go_file: Path, is_human: bool = True, 
                sgd_to_uniprot: Optional[Dict[str, str]] = None) -> pd.DataFrame:
    """Charge les annotations GO et convertit les identifiants en UniProt si nécessaire"""
    try:
        if is_human:
            # Chargement des données humaines (déjà en UniProt)
            df = pd.read_csv(go_file, sep='\t', usecols=['Entry', 'Gene Ontology (GO)'], 
                            dtype={'Entry': 'string'})
            
            # Extraction des termes GO
            go_data = []
            for entry, go_str in df.dropna(subset=['Gene Ontology (GO)']).itertuples(index=False):
                terms = re.findall(r'GO:\d+', go_str)
                go_data.extend([(entry, term) for term in terms])
            
            df_go = pd.DataFrame(go_data, columns=['protein', 'go_term'])
        else:
            # Chargement des données de levure (SGD -> besoin de conversion)
            df = pd.read_csv(go_file, sep='\t', header=None, 
                           usecols=[2, 5], names=['protein', 'go_term'],
                           dtype={'protein': 'string', 'go_term': 'string'})
            
            # Filtrage des termes GO valides
            df_go = df[df['go_term'].str.contains(r'^GO:\d+$', na=False)].copy()
            
            # Conversion SGD -> UniProt si le mapping est fourni
            if sgd_to_uniprot:
                logger.info("Converting SGD IDs to UniProt IDs in GO annotations")
                df_go['protein'] = df_go['protein'].map(sgd_to_uniprot)
                df_go = df_go.dropna(subset=['protein'])  # Enlever les non mappés
        
        logger.info(f"Loaded {len(df_go)} GO annotations")
        return df_go.drop_duplicates().reset_index(drop=True)
    except Exception as e:
        logger.error(f"Erreur lors du chargement des annotations GO: {e}")
        return pd.DataFrame(columns=['protein', 'go_term'])

def calculate_functional_similarity(proteins: list, go_annotations: pd.DataFrame) -> Optional[pd.DataFrame]:
    """Calcule la similarité fonctionnelle entre les protéines"""
    try:
        proteins_set = sorted(set(proteins))
        logger.info(f"Calculating functional similarity for {len(proteins_set)} unique proteins")
        
        # Filtrer les annotations pour les protéines d'intérêt
        go_filtered = go_annotations[go_annotations['protein'].isin(proteins_set)].copy()
        
        if len(go_filtered) == 0:
            logger.warning("No matching GO annotations found for the proteins")
            return None
        
        # Créer une matrice protéine-terme GO
        protein_cat = pd.CategoricalDtype(categories=proteins_set, ordered=True)
        go_filtered['protein'] = go_filtered['protein'].astype(protein_cat)
        
        # Construction de la matrice creuse
        row_ind = go_filtered['protein'].cat.codes
        col_ind = pd.Categorical(go_filtered['go_term']).codes
        data = np.ones(len(go_filtered), dtype=np.int8)
        
        sparse_matrix = sparse.csr_matrix(
            (data, (row_ind, col_ind)),
            shape=(len(proteins_set), len(go_filtered['go_term'].unique()))
        )
        # Calcul de similarité cosinus
        logger.info("Computing cosine similarity...")
        similarity_matrix = cosine_similarity(sparse_matrix)
        
        return pd.DataFrame(similarity_matrix, 
                          index=proteins_set, 
                          columns=proteins_set)
    except Exception as e:
        logger.error(f"Error calculating functional similarity: {e}")
        return None

def load_ppi_network(ppi_file: Path) -> pd.DataFrame:
    """Charge un réseau PPI (supposé utiliser des UniProt IDs)"""
    try:
        ppi = pd.read_csv(ppi_file, sep='\t', header=None,
                         names=['protein1', 'protein2'], dtype='string')
        
        # Nettoyage des identifiants
        ppi['protein1'] = ppi['protein1'].str.upper().str.strip()
        ppi['protein2'] = ppi['protein2'].str.upper().str.strip()
        
        # Suppression des doublons et des valeurs manquantes
        ppi = ppi.dropna().drop_duplicates()
        
        logger.info(f"Loaded PPI network with {len(ppi)} interactions")
        return ppi
    except Exception as e:
        logger.error(f"Error loading PPI network: {e}")
        return pd.DataFrame(columns=['protein1', 'protein2'])

def main():
    # Chargement des mappings
    uniprot_to_sgd, sgd_to_uniprot = load_mapping(MAPPING_FILE)
    
    # Configuration des réseaux à traiter
    networks = {
        "STRING_levure": (GO_SLIM_YEAST, False),
        "BIOGRID_humain": (GO_SLIM_HUMAN, True),
        "STRING_humain": (GO_SLIM_HUMAN, True),
        "BIOGRID_levure": (GO_SLIM_YEAST, False),
        "DIP_levure": (GO_SLIM_YEAST, False)   
    }

    for network_name, (go_file, is_human) in networks.items():
        logger.info(f"\nProcessing {network_name} network")
        
        # Chargement du réseau PPI
        ppi_file = INTERACTIONS_DIR / f"{network_name}.txt"
        ppi = load_ppi_network(ppi_file)
        
        if ppi.empty:
            logger.warning(f"No valid interactions found for {network_name}")
            continue
        
        # Chargement des annotations GO avec conversion si nécessaire
        go_annotations = load_go_slim(
            go_file, 
            is_human=is_human,
            sgd_to_uniprot=sgd_to_uniprot if not is_human else None
        )
        
        if go_annotations.empty:
            logger.warning(f"No GO annotations found for {network_name}")
            continue
        
        # Liste de toutes les protéines du réseau
        all_proteins = list(set(ppi['protein1']).union(set(ppi['protein2'])))
        
        # Calcul de la similarité fonctionnelle
        similarity_matrix = calculate_functional_similarity(all_proteins, go_annotations)
        
        if similarity_matrix is None:
            logger.warning(f"Could not compute similarity matrix for {network_name}")
            continue
        
        # Ajout des poids aux interactions
        ppi['weight'] = ppi.apply(
            lambda row: similarity_matrix.loc[row['protein1'], row['protein2']]
            if row['protein1'] in similarity_matrix.index and row['protein2'] in similarity_matrix.columns
            else 0.0,
            axis=1
        )
        
        # Filtrage des interactions avec poids > 0
        ppi = ppi[ppi['weight'] > 0]
        
        # Sauvegarde
        output_file = OUTPUT_DIR / f"FS_{network_name}.txt"
        ppi.to_csv(output_file, sep='\t', index=False)
        logger.info(f"Saved weighted network to {output_file} with {len(ppi)} interactions")

if __name__ == "__main__":
    main()

2025-04-17 17:36:53,838 - INFO - Loaded 6726 UniProt to SGD mappings
2025-04-17 17:36:53,873 - INFO - 
Processing STRING_levure network
2025-04-17 17:36:55,126 - INFO - Loaded PPI network with 96810 interactions
2025-04-17 17:36:55,372 - INFO - Converting SGD IDs to UniProt IDs in GO annotations
2025-04-17 17:36:55,408 - INFO - Loaded 48478 GO annotations
2025-04-17 17:36:55,761 - INFO - Calculating functional similarity for 5707 unique proteins
2025-04-17 17:36:55,859 - INFO - Computing cosine similarity...
2025-04-17 17:37:37,866 - INFO - Saved weighted network to C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\autres\FS_STRING_levure.txt with 86142 interactions
2025-04-17 17:37:37,868 - INFO - 
Processing BIOGRID_humain network
2025-04-17 17:37:38,127 - INFO - Loaded PPI network with 88647 interactions
2025-04-17 17:37:42,871 - INFO - Loaded 764609 GO annotations
2025-04-17 17:37:44,222 - INFO - Calculating functional similarity for 11266 unique proteins
2025-04-17 17:37:4

# 2. Co-Expression similarity:

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import os
from collections import defaultdict
import glob

def load_mapping(file_path):
    """Charge les mappings d'identifiants avec tous les types pertinents"""
    mapping = {
        'uniprot_to_gene': defaultdict(list),
        'uniprot_to_name': defaultdict(list),
        'gene_to_uniprot': defaultdict(list),
        'name_to_uniprot': defaultdict(list),
        'orf_to_uniprot': defaultdict(list),
        'uniprot_to_uniprot': defaultdict(list),
        'ensembl_to_uniprot': defaultdict(list),
        'string_to_uniprot': defaultdict(list)
    }
    
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 3:
                uniprot_id = parts[0]
                db = parts[1]
                db_id = parts[2]
                
                # Mapper Uniprot à lui-même
                mapping['uniprot_to_uniprot'][uniprot_id].append(uniprot_id)
                
                if db == 'Gene_OrderedLocusName':
                    mapping['uniprot_to_gene'][uniprot_id].append(db_id)
                    mapping['gene_to_uniprot'][db_id].append(uniprot_id)
                elif db == 'Gene_Name':
                    mapping['uniprot_to_name'][uniprot_id].append(db_id)
                    mapping['name_to_uniprot'][db_id.upper()].append(uniprot_id)
                elif db == 'ORF':
                    mapping['orf_to_uniprot'][db_id].append(uniprot_id)
                elif db == 'Ensembl':
                    mapping['ensembl_to_uniprot'][db_id].append(uniprot_id)
                elif db == 'STRING':
                    mapping['string_to_uniprot'][db_id].append(uniprot_id)
    
    # Simplifier les mappings en gardant le premier élément
    simple_mappings = {}
    for map_type in mapping:
        simple_mappings[map_type] = {k: v[0] for k, v in mapping[map_type].items() if v}
    
    return simple_mappings

def load_expression_data(file_path, mappings, species):
    """Charge les données d'expression en fonction de l'espèce"""
    try:
        # Trouver le début des données
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith('!dataset_table_begin'):
                    break
            
            # Lire les données avec toutes les colonnes
            df = pd.read_csv(f, sep='\t', dtype=str, low_memory=False)
        
        # Extraire les colonnes d'expression (commençant par GSM)
        expression_cols = [col for col in df.columns if col.startswith('GSM')]
        
        # Vérifier que toutes les colonnes GSM ont le même nombre de valeurs
        gsm_lengths = {col: len(df[col].dropna()) for col in expression_cols}
        if len(set(gsm_lengths.values())) > 1:
            print("Avertissement: Les colonnes GSM ont des longueurs différentes!")
            # Prendre le nombre minimal de valeurs
            min_length = min(gsm_lengths.values())
            for col in expression_cols:
                df[col] = df[col].dropna().head(min_length)
        
        # Garder seulement les colonnes nécessaires
        df = df[['IDENTIFIER'] + expression_cols].dropna()
        df = df.set_index('IDENTIFIER')
        
        # Convertir en numérique en gérant les erreurs
        for col in expression_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna(how='all')
        
        # Mapping différent selon l'espèce
        if species == 'human':
            # Pour les données humaines, essayer plusieurs stratégies
            def human_mapper(x):
                try:
                    x = str(x).upper()
                    if x in mappings['name_to_uniprot']:
                        return mappings['name_to_uniprot'][x]
                    return None
                except:
                    return None
            
            df.index = df.index.map(human_mapper)
        else:
            # Pour la levure, utiliser les ORFs/noms de gènes
            def yeast_mapper(x):
                try:
                    x = str(x)
                    if x in mappings['gene_to_uniprot']:
                        return mappings['gene_to_uniprot'][x]
                    if x in mappings['orf_to_uniprot']:
                        return mappings['orf_to_uniprot'][x]
                    if x.upper() in mappings['name_to_uniprot']:
                        return mappings['name_to_uniprot'][x.upper()]
                    return None
                except:
                    return None
            
            df.index = df.index.map(yeast_mapper)
        
        return df.dropna()
    except Exception as e:
        print(f"Erreur lors du chargement des données d'expression: {str(e)}")
        return pd.DataFrame()

def calculate_pcc(v, u):
    """Calcule le coefficient de corrélation de Pearson avec vérification de longueur"""
    try:
        # Convertir en arrays numpy et s'assurer qu'ils sont 1D
        v = np.asarray(v, dtype=float).flatten()
        u = np.asarray(u, dtype=float).flatten()
        
        # Trouver la longueur minimale
        min_len = min(len(v), len(u))
        if min_len < 2:
            return np.nan
            
        # Tronquer les vecteurs à la même longueur
        v = v[:min_len]
        u = u[:min_len]
        
        # Masque pour les valeurs manquantes
        mask = ~np.isnan(v) & ~np.isnan(u)
        v_clean = v[mask]
        u_clean = u[mask]
        
        if len(v_clean) < 2:
            return np.nan
        
        # Vérifier si l'un des vecteurs est constant
        if np.all(v_clean == v_clean[0]) or np.all(u_clean == u_clean[0]):
            return np.nan
            
        pcc, _ = pearsonr(v_clean, u_clean)
        return (pcc + 1) / 2  # Transformation vers [0,1]
    except Exception as e:
        print(f"Error calculating PCC between shapes {np.shape(v)} and {np.shape(u)}: {str(e)}")
        return np.nan

def calculate_coexpression(ppi, expr_data, output_file):
    """Calcule et sauvegarde les scores de co-expression"""
    results = []
    proteins_in_ppi = set(ppi['protein1']).union(set(ppi['protein2']))
    proteins_in_expr = set(expr_data.index)
    common_proteins = proteins_in_ppi.intersection(proteins_in_expr)
    
    print(f"\nRecouvrement entre PPI et expression:")
    print(f"- Protéines dans PPI: {len(proteins_in_ppi)}")
    print(f"- Protéines dans expression: {len(proteins_in_expr)}")
    print(f"- Protéines communes: {len(common_proteins)}")
    
    if len(common_proteins) < 2:
        print("Avertissement: Pas assez de protéines communes pour calculer la co-expression!")
        return
    
    # Pré-traitement: s'assurer que toutes les protéines ont le même nombre de points de données
    min_length = min([len(expr_data.loc[p].values) for p in common_proteins])
    expr_data = expr_data.apply(lambda x: x[:min_length], axis=1)
    
    for _, row in ppi.iterrows():
        p1, p2 = row['protein1'], row['protein2']
        if p1 in expr_data.index and p2 in expr_data.index:
            # Get expression values and ensure they're 1D arrays
            v = expr_data.loc[p1].values
            u = expr_data.loc[p2].values
            
            # Flatten arrays in case they're 2D
            v = np.asarray(v).flatten()
            u = np.asarray(u).flatten()
            
            # Skip if one array is empty
            if len(v) == 0 or len(u) == 0:
                continue
                
            pcc = calculate_pcc(v, u)
            if not np.isnan(pcc):
                results.append({'protein1': p1, 'protein2': p2, 'PCC': pcc})
    
    if results:
        result_df = pd.DataFrame(results)
        result_df.to_csv(output_file, sep='\t', index=False)
        print(f"\nRésultats sauvegardés dans {output_file} ({len(result_df)} paires valides)")
    else:
        print("\nAucun résultat valide à sauvegarder.")

def process_dataset(base_dir, species):
    """Traite un ensemble de données complet"""
    print(f"\n=== Traitement des données {species} ===")
    
    # Déterminer les noms de fichiers en fonction de l'espèce
    if species == 'human':
        mapping_file = os.path.join(base_dir, "raw data/autres/HUMAN_9606_idmapping.dat")
        expr_file = os.path.join(base_dir, "raw data/autres/human_co-expression.soft")
        ppi_pattern = os.path.join(base_dir, "clean data/interactions/*humain*")
    else:  # levure
        mapping_file = os.path.join(base_dir, "raw data/autres/YEAST_559292_idmapping.dat")
        expr_file = os.path.join(base_dir, "raw data/autres/levure_co-expression.soft")
        ppi_pattern = os.path.join(base_dir, "clean data/interactions/*levure*")
    
    # Charger les mappings
    if not os.path.exists(mapping_file):
        print(f"Fichier de mapping introuvable: {mapping_file}")
        return
    
    mappings = load_mapping(mapping_file)
    print(f"\nStatistiques de mapping:")
    print(f"- Uniprot IDs: {len(mappings['uniprot_to_uniprot'])}")
    print(f"- Gene names: {len(mappings['name_to_uniprot'])}")
    print(f"- Gene IDs: {len(mappings['gene_to_uniprot'])}")
    print(f"- ORFs: {len(mappings['orf_to_uniprot'])}")
    print(f"- Ensembl IDs: {len(mappings['ensembl_to_uniprot'])}")
    print(f"- STRING IDs: {len(mappings['string_to_uniprot'])}")
    
    # Charger les données d'expression
    if not os.path.exists(expr_file):
        print(f"Fichier d'expression introuvable: {expr_file}")
        return
    
    expr_data = load_expression_data(expr_file, mappings, species)
    if expr_data.empty:
        print("Avertissement: Aucune donnée d'expression valide après mapping!")
        return
    
    print(f"\nDonnées d'expression chargées: {len(expr_data)} protéines")
    
    # Traiter les fichiers PPI
    ppi_files = glob.glob(ppi_pattern)
    if not ppi_files:
        print(f"Aucun fichier PPI trouvé avec le pattern: {ppi_pattern}")
    
    for ppi_file in ppi_files:
        print(f"\nTraitement de {os.path.basename(ppi_file)}")
        try:
            ppi = pd.read_csv(ppi_file, sep='\t', header=None, names=['protein1', 'protein2'], dtype=str)
            
            # Mapper les identifiants
            ppi['protein1'] = ppi['protein1'].apply(lambda x: mappings['uniprot_to_uniprot'].get(x, None))
            ppi['protein2'] = ppi['protein2'].apply(lambda x: mappings['uniprot_to_uniprot'].get(x, None))
            ppi = ppi.dropna()
            
            if ppi.empty:
                print("Avertissement: Aucune interaction valide après mapping!")
                continue
            
            print(f"Interactions chargées: {len(ppi)}")
            
            # Calculer la co-expression
            output_name = os.path.splitext(os.path.basename(ppi_file))[0]
            output_file = os.path.join(base_dir, f"clean data/autres/coexpression_{output_name}.txt")
            
            calculate_coexpression(ppi, expr_data, output_file)
        except Exception as e:
            print(f"Erreur lors du traitement: {str(e)}")

def main():
    base_dir = "C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data"
    
    # Créer le dossier de résultats s'il n'existe pas
    os.makedirs(os.path.join(base_dir, "clean data/autres"), exist_ok=True)
    
    # Traiter les deux espèces
    process_dataset(base_dir, 'human')
    process_dataset(base_dir, 'levure')

if __name__ == "__main__":
    main()


=== Traitement des données human ===

Statistiques de mapping:
- Uniprot IDs: 237779
- Gene names: 26246
- Gene IDs: 0
- ORFs: 0
- Ensembl IDs: 24400
- STRING IDs: 19110

Données d'expression chargées: 28869 protéines

Traitement de BIOGRID_humain.txt
Interactions chargées: 87806

Recouvrement entre PPI et expression:
- Protéines dans PPI: 11217
- Protéines dans expression: 17721
- Protéines communes: 10423

Résultats sauvegardés dans C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data\clean data/autres/coexpression_BIOGRID_humain.txt (72599 paires valides)

Traitement de STRING_humain.txt
Interactions chargées: 228171

Recouvrement entre PPI et expression:
- Protéines dans PPI: 15943
- Protéines dans expression: 17721
- Protéines communes: 14415

Résultats sauvegardés dans C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data\clean data/autres/coexpression_STRING_humain.txt (182177 paires valides)

=== Traitement des données levure ===

Statistiques de mapping:
- Uniprot IDs: 6900
- Gene names: 

# 3. Subcellular localization:

In [4]:
import pandas as pd
import logging
from pathlib import Path

# Configuration du logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler('ppi_weighting.log'),
                        logging.StreamHandler()
                    ])

def read_interaction_file(file_path):
    """Lire un fichier d'interactions PPI"""
    try:
        df = pd.read_csv(file_path, sep='\t', header=None, usecols=[0, 1],
                         dtype={0: str, 1: str})
        df.columns = ['protein1', 'protein2']
        df['protein1'] = df['protein1'].str.strip().str.upper()
        df['protein2'] = df['protein2'].str.strip().str.upper()
        if df.empty:
            logging.warning(f"Aucune interaction trouvée dans {file_path}")
            return pd.DataFrame()
        logging.info(f"{len(df)} interactions chargées depuis {file_path}")
        return df.drop_duplicates()
    except Exception as e:
        logging.error(f"Erreur lecture {file_path} : {str(e)}")
        return pd.DataFrame()

def read_compartment_file(file_path):
    """Lire et préparer les données de compartimentation (avec Mapped_UniProtKB_ID)"""
    try:
        df = pd.read_csv(file_path, sep='\t', 
                         usecols=['Mapped_UniProtKB_ID', 'GO_ID', 'Mapping_Status'],
                         dtype=str)
        df = df[df['Mapping_Status'] == 'Success'].copy()
        df['Mapped_UniProtKB_ID'] = df['Mapped_UniProtKB_ID'].str.strip().str.upper()
        df['GO_ID'] = df['GO_ID'].str.strip()
        if df.empty:
            logging.warning(f"Aucune donnée valide dans {file_path}")
            return pd.DataFrame()
        logging.info(f"{len(df)} entrées de compartiment valides chargées depuis {file_path}")
        return df
    except Exception as e:
        logging.error(f"Erreur lecture {file_path} : {str(e)}")
        return pd.DataFrame()

def calculate_sl_similarity(prot1_comps, prot2_comps):
    """Calculer la similarité de localisation subcellulaire"""
    intersection = len(prot1_comps & prot2_comps)
    if intersection == 0:
        return 0.0
    sl_v = len(prot1_comps)
    sl_u = len(prot2_comps)
    return (intersection ** 2) / (sl_v * sl_u)

def calculate_weights(interaction_df, compartment_df):
    """Calculer les poids basés sur la similarité des compartiments"""
    if interaction_df.empty or compartment_df.empty:
        return pd.DataFrame()
    
    # Dictionnaire des GO_IDs par protéine (ID = Mapped_UniProtKB_ID)
    protein_compartments = compartment_df.groupby('Mapped_UniProtKB_ID')['GO_ID'].apply(set).to_dict()

    results = []
    missing_proteins = set()

    for _, row in interaction_df.iterrows():
        prot1, prot2 = row['protein1'], row['protein2']
        if prot1 not in protein_compartments or prot2 not in protein_compartments:
            if prot1 not in protein_compartments:
                missing_proteins.add(prot1)
            if prot2 not in protein_compartments:
                missing_proteins.add(prot2)
            continue
        sim = calculate_sl_similarity(protein_compartments[prot1], protein_compartments[prot2])
        if sim > 0:
            results.append({'protein1': prot1, 'protein2': prot2, 'weight': sim})

    if missing_proteins:
        logging.warning(f"{len(missing_proteins)} protéines sans données de compartiment")

    return pd.DataFrame(results)

def process_species_networks(species_files, compartment_files):
    """Traiter les réseaux PPI pour chaque espèce"""
    for species, species_file in species_files.items():
        logging.info(f"\n{'='*50}")
        logging.info(f"Traitement des réseaux pour {species}")
        logging.info(f"{'='*50}")

        # Charger les données de compartiment
        compartment_df = read_compartment_file(compartment_files[species])
        if compartment_df.empty:
            logging.error(f"Impossible de traiter {species} - données de compartiment vides")
            continue

        for network_name, network_file in species_file.items():
            logging.info(f"\nTraitement du réseau {network_name}...")
            interaction_df = read_interaction_file(network_file)
            if interaction_df.empty:
                logging.warning("Aucune interaction valide trouvée")
                continue

            weighted_df = calculate_weights(interaction_df, compartment_df)
            if weighted_df.empty:
                logging.warning(f"Aucune interaction valide avec poids pour {network_name}")
                continue

            output_file = r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\autres\\"+f"SL_{network_name}_{species}.txt"
            weighted_df.to_csv(output_file, sep='\t', index=False, float_format='%.6f')
            logging.info(f"Réseau pondéré sauvegardé ({len(weighted_df)} interactions)")

def main():
    BASE_DIR = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data")

    species_files = {
        "levure": {
            "STRING": BASE_DIR / "interactions/STRING_levure.txt",
            "BIOGRID": BASE_DIR / "interactions/BIOGRID_levure.txt",
            "DIP": BASE_DIR / "interactions/DIP_levure.txt"
        },
        "humain": {
            "STRING": BASE_DIR / "interactions/STRING_humain.txt",
            "BIOGRID": BASE_DIR / "interactions/BIOGRID_humain.txt"
        }
    }

    compartment_files = {
        "levure": BASE_DIR / "autres/yeast_compartment_mapped.txt",
        "humain": BASE_DIR / "autres/human_compartment_mapped.txt"
    }

    process_species_networks(species_files, compartment_files)

if __name__ == "__main__":
    main()


2025-04-17 20:10:00,926 - INFO - 
2025-04-17 20:10:00,934 - INFO - Traitement des réseaux pour levure
2025-04-17 20:10:00,941 - INFO - ==================================================
2025-04-17 20:10:02,685 - INFO - 717525 entrées de compartiment valides chargées depuis C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\autres\yeast_compartment_mapped.txt
2025-04-17 20:10:02,685 - INFO - 
Traitement du réseau STRING...
2025-04-17 20:10:02,835 - INFO - 96810 interactions chargées depuis C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\STRING_levure.txt
2025-04-17 20:10:12,228 - WARNING - 285 protéines sans données de compartiment
2025-04-17 20:10:12,596 - INFO - Réseau pondéré sauvegardé (91378 interactions)
2025-04-17 20:10:12,596 - INFO - 
Traitement du réseau BIOGRID...
2025-04-17 20:10:12,959 - INFO - 300000 interactions chargées depuis C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\interactions\BIOGRID_levure.txt
2025-04-17 20:10:31,991 - 

# 4. Common Neighbor similarity:

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import os
import glob

def load_ppi_network(file_path):
    """Charge un réseau PPI à partir d'un fichier"""
    ppi = pd.read_csv(file_path, sep='\t', header=None, names=['protein1', 'protein2'])
    return ppi

def build_graph(ppi_df):
    """Construit un graphe NetworkX à partir d'un dataframe PPI"""
    G = nx.Graph()
    for _, row in ppi_df.iterrows():
        G.add_edge(row['protein1'], row['protein2'])
    return G

def calculate_hcn_similarity(G, output_file):
    """Calcule la similarité HCN pour toutes les paires de protéines connectées"""
    results = []
    
    # Pré-calculer les voisins pour tous les nœuds
    neighbors = {node: set(G.neighbors(node)) for node in G.nodes()}
    
    for edge in G.edges():
        v, u = edge
        N_v = neighbors[v]
        N_u = neighbors[u]
        
        # Calculer les composantes de la formule HCN
        NCN = N_v & N_u  # Intersection des voisins (common neighbors)
        N_union = N_v | N_u  # Union des voisins
        
        # Calculer chaque terme de la formule
        numerator = len(NCN)**2
        denominator = (len(N_v) * len(N_u) * len(N_union))
        
        # Éviter la division par zéro
        if denominator == 0:
            hcn = 0.0
        else:
            hcn = numerator / denominator
        
        results.append({
            'protein1': v,
            'protein2': u,
            'HCN_score': hcn,
        })
    
    # Sauvegarder les résultats avec seulement les 3 premières colonnes
    result_df = pd.DataFrame(results)
    result_df = result_df[['protein1', 'protein2', 'HCN_score']]  # Garder seulement les 3 premières colonnes
    result_df.to_csv(output_file, sep='\t', index=False)
    print(f"Résultats HCN sauvegardés dans {output_file}")
    return result_df

def process_all_ppi_networks(base_dir, output_dir):
    """Traite tous les fichiers PPI dans le dossier spécifié"""
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Trouver tous les fichiers PPI
    ppi_files = glob.glob(os.path.join(base_dir, "*i*"))[:5]  # Prendre les 5 premiers fichiers
    
    for ppi_file in ppi_files:
        print(f"\nTraitement du fichier: {os.path.basename(ppi_file)}")
        
        # Charger le réseau PPI
        ppi_df = load_ppi_network(ppi_file)
        print(f"Nombre d'interactions chargées: {len(ppi_df)}")
        
        # Construire le graphe
        G = build_graph(ppi_df)
        print(f"Nombre de nœuds: {G.number_of_nodes()}")
        print(f"Nombre d'arêtes: {G.number_of_edges()}")
        
        # Calculer les scores HCN
        output_name = os.path.splitext(os.path.basename(ppi_file))[0]
        output_file = os.path.join(output_dir, f"HCN_scores_{output_name}.txt")
        
        hcn_results = calculate_hcn_similarity(G, output_file)
        
        # Afficher quelques statistiques
        print(f"Score HCN moyen: {hcn_results['HCN_score'].mean():.4f}")
        print(f"Nombre moyen de voisins communs: {hcn_results['HCN_score'].count()}")

if __name__ == "__main__":
    # Configuration des chemins
    base_dir = "C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data/clean data/interactions"
    output_dir = "C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data/clean data/autres"
    
    # Exécuter le traitement pour tous les réseaux PPI
    process_all_ppi_networks(base_dir, output_dir)



Traitement du fichier: BIOGRID_humain.txt
Nombre d'interactions chargées: 88647
Nombre de nœuds: 11266
Nombre d'arêtes: 88647
Résultats HCN sauvegardés dans C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data/clean data/autres\HCN_scores_BIOGRID_humain.txt
Score HCN moyen: 0.0011
Nombre moyen de voisins communs: 88647

Traitement du fichier: BIOGRID_levure.txt
Nombre d'interactions chargées: 300000
Nombre de nœuds: 5805
Nombre d'arêtes: 300000
Résultats HCN sauvegardés dans C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data/clean data/autres\HCN_scores_BIOGRID_levure.txt
Score HCN moyen: 0.0001
Nombre moyen de voisins communs: 300000

Traitement du fichier: DIP_levure.txt
Nombre d'interactions chargées: 22614
Nombre de nœuds: 5144
Nombre d'arêtes: 22614
Résultats HCN sauvegardés dans C:/Users/PC/Documents/M2 HPC/PFE/PFE_CODE/Data/clean data/autres\HCN_scores_DIP_levure.txt
Score HCN moyen: 0.0011
Nombre moyen de voisins communs: 22614

Traitement du fichier: STRING_humain.txt
Nombre d'interact

# 5. Weighted PPI Network:

In [6]:
import pandas as pd
from pathlib import Path
import logging
from typing import Dict, List, Set, Tuple
import numpy as np

# Configuration des chemins
BASE_DIR = Path(r"C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data")
CLEAN_DATA_DIR = BASE_DIR / "clean data"
OUTPUT_DIR = CLEAN_DATA_DIR / "autres"
WEIGHTED_DIR = CLEAN_DATA_DIR / "weighted_networks"

# Création des dossiers
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WEIGHTED_DIR.mkdir(parents=True, exist_ok=True)

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(WEIGHTED_DIR / 'network_stats.log')
    ]
)
logger = logging.getLogger(__name__)

def calculate_network_density(df: pd.DataFrame) -> float:
    """Calcule la densité du réseau (non dirigé)"""
    if df.empty:
        return 0.0
    
    # Nombre de noeuds uniques
    unique_nodes = pd.unique(np.concatenate([df['protein1'], df['protein2']]))
    num_nodes = len(unique_nodes)
    
    # Nombre d'arêtes existantes
    num_edges = len(df)
    
    # Nombre maximum possible d'arêtes dans un graphe non dirigé
    max_possible_edges = num_nodes * (num_nodes - 1) / 2
    
    # Éviter la division par zéro
    if max_possible_edges == 0:
        return 0.0
    
    # Calcul de la densité
    density = num_edges / max_possible_edges
    
    return density

def load_weighted_network(network_name: str, species: str, weight_type: str) -> pd.DataFrame:
    """Charge un réseau pondéré spécifique de manière optimisée"""
    file_patterns = {
        'FS': f"FS_{network_name}{species}.txt",
        'SL': f"SL_{network_name}{species}.txt", 
        'CO': f"coexpression_{network_name}{species}.txt",
        'HCN': f"HCN_scores_{network_name}{species}.txt"
    }
    
    file_path = OUTPUT_DIR / file_patterns.get(weight_type, "")
    try:
        df = pd.read_csv(file_path, sep='\t', dtype={'protein1': 'str', 'protein2': 'str', 'weight': 'float64'}, 
                         low_memory=False)
        df.columns = df.columns.str.lower()
        
        if 'weight' not in df.columns and len(df.columns) >= 3:
            weight_col = df.columns[2]
            df = df.rename(columns={weight_col: 'weight'})
            df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        
        df = df.dropna(subset=['protein1', 'protein2', 'weight'])
        df = df.drop_duplicates(subset=['protein1', 'protein2'])
        
        mask = df['protein1'] > df['protein2']
        df.loc[mask, ['protein1', 'protein2']] = df.loc[mask, ['protein2', 'protein1']].values
        
        return df[['protein1', 'protein2', 'weight']]
    except Exception as e:
        logger.error(f"Erreur de chargement {file_path}: {e}")
        return pd.DataFrame()

def get_common_interactions(networks: Dict[str, pd.DataFrame]) -> Set[Tuple[str, str]]:
    """Trouve les interactions communes aux 4 réseaux"""
    if not networks:
        return set()
    
    pair_sets = []
    for wt, df in networks.items():
        pairs = set(zip(df['protein1'], df['protein2']))
        pair_sets.append(pairs)
        logger.info(f"{wt}: {len(pairs)} interactions")
    
    common_pairs = set.intersection(*pair_sets)
    logger.info(f"Interactions communes: {len(common_pairs)}")
    
    return common_pairs

def calculate_final_network(common_pairs: Set[Tuple[str, str]], 
                          networks: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Calcule le réseau final avec poids moyens"""
    common_df = pd.DataFrame(list(common_pairs), columns=['protein1', 'protein2'])
    
    weight_data = {
        'protein1': common_df['protein1'],
        'protein2': common_df['protein2'],
        'weights': np.zeros(len(common_df))
    }
    
    count = np.zeros(len(common_df))
    
    for df in networks.values():
        merged = pd.merge(common_df, df, on=['protein1', 'protein2'], how='left')
        valid_weights = merged['weight'].fillna(0).values
        weight_data['weights'] += valid_weights
        count += (valid_weights != 0)
    
    mean_weights = np.divide(weight_data['weights'], count, where=(count != 0))
    
    final_network = pd.DataFrame({
        'protein1': weight_data['protein1'],
        'protein2': weight_data['protein2'],
        'mean_weight': mean_weights
    })
    
    final_network = final_network[count > 0]
    
    return final_network

def analyze_final_network(df: pd.DataFrame, network_name: str) -> Dict[str, float]:
    """Analyse le réseau final et retourne les statistiques"""
    if df.empty:
        return {}
    
    unique_proteins = pd.unique(np.concatenate([df['protein1'], df['protein2']])).size
    num_interactions = len(df)
    mean_w = df['mean_weight'].mean()
    density = calculate_network_density(df)
    
    logger.info(f"\nRésultats pour {network_name}:")
    logger.info(f"- Protéines uniques: {unique_proteins}")
    logger.info(f"- Interactions: {num_interactions}")
    logger.info(f"- Poids moyen: {mean_w:.4f}")
    logger.info(f"- Densité du réseau: {density:.6f}")
    
    return {
        'network': network_name,
        'unique_proteins': unique_proteins,
        'interactions': num_interactions,
        'mean_weight': mean_w,
        'density': density
    }

def process_single_network(network_name: str, species: str) -> Dict[str, float]:
    """Traite un seul réseau PPI"""
    logger.info(f"\n{'='*40}\nTraitement de {network_name} ({species})\n{'='*40}")
    
    weighted_types = ['FS', 'SL', 'CO', 'HCN']
    networks = {wt: load_weighted_network(network_name, species, wt) for wt in weighted_types}
    
    if any(df.empty for df in networks.values()):
        missing = [wt for wt, df in networks.items() if df.empty]
        logger.warning(f"Réseaux manquants: {missing}")
        return {}
    
    common_pairs = get_common_interactions(networks)
    if not common_pairs:
        logger.warning("Aucune interaction commune trouvée")
        return {}
    
    final_network = calculate_final_network(common_pairs, networks)
    
    if final_network.empty:
        logger.warning("Aucune interaction valide après calcul des poids moyens")
        return {}
    
    output_file = WEIGHTED_DIR / f"weighted_{network_name}{species}.txt"
    final_network.to_csv(output_file, sep='\t', index=False)
    logger.info(f"\nRéseau final sauvegardé dans {output_file}")
    
    return analyze_final_network(final_network, f"{network_name}_{species}")

def main():
    network_species_pairs = [
        ("BIOGRID_", "humain"),
        ("STRING_", "humain"),
        ("BIOGRID_", "levure"),
        ("DIP_", "levure"),
        ("STRING_", "levure")
    ]
    
    all_stats = []
    
    for network_name, species in network_species_pairs:
        stats = process_single_network(network_name, species)
        if stats:
            all_stats.append(stats)
    
    logger.info("\n\nRécapitulatif final:")
    stats_df = pd.DataFrame(all_stats)
    logger.info("\n" + stats_df.to_string(index=False, float_format="%.6f"))
    
    stats_file = OUTPUT_DIR / "network_statistics.txt"
    stats_df.to_csv(stats_file, sep='\t', index=False, float_format="%.6f")
    logger.info(f"\nStatistiques sauvegardées dans {stats_file}")

if __name__ == "__main__":
    main()

2025-04-17 20:17:24,467 - INFO - 
Traitement de BIOGRID_ (humain)
2025-04-17 20:17:25,005 - INFO - FS: 80778 interactions
2025-04-17 20:17:25,042 - INFO - SL: 86981 interactions
2025-04-17 20:17:25,088 - INFO - CO: 72599 interactions
2025-04-17 20:17:25,139 - INFO - HCN: 88647 interactions
2025-04-17 20:17:25,188 - INFO - Interactions communes: 66373
2025-04-17 20:17:25,687 - INFO - 
Réseau final sauvegardé dans C:\Users\PC\Documents\M2 HPC\PFE\PFE_CODE\Data\clean data\weighted_networks\weighted_BIOGRID_humain.txt
2025-04-17 20:17:25,708 - INFO - 
Résultats pour BIOGRID__humain:
2025-04-17 20:17:25,708 - INFO - - Protéines uniques: 9171
2025-04-17 20:17:25,708 - INFO - - Interactions: 66373
2025-04-17 20:17:25,708 - INFO - - Poids moyen: 0.2936
2025-04-17 20:17:25,721 - INFO - - Densité du réseau: 0.001578
2025-04-17 20:17:25,725 - INFO - 
Traitement de STRING_ (humain)
2025-04-17 20:17:27,119 - INFO - FS: 205424 interactions
2025-04-17 20:17:27,219 - INFO - SL: 227814 interactions
202